<a href="https://colab.research.google.com/github/UnnatiChitra/Business-Chat-Bot/blob/main/Business_Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pinecone-client cohere openai PyPDF2 langchain-cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Starting-a-Business.pdf to Starting-a-Business.pdf


In [ ]:
# provide the path of  pdf file/files.
data = PdfReader('/content/Starting-a-Business.pdf')

In [ ]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(data.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
raw_text

'A Blue Beetle Books Publication\nStarting a Business 101   \n(Canadian Edition)\nCopyright © 2012 Blue Beetle Books\nStarting a Business 101\n(Canadian Edition)  \nPublished as an eBook original by  \nBlue Beetle Books.\nNo part of this eBook may be reproduced in  \nany manner whatsoever without the written  \npermission of Blue Beetle Books.\nBlue Beetle Books\n1030 Boeing Close, Langford BC V9B 0R2\nTel: 250-704-6686\nE: info@bluebeetlebooks.com\nwww.bluebeetlebooks.com3\nwww.pacf.caPrince Albert and District Community  \nFutures Development CorporationStarting a Business 101   \n(Canadian Edition)\nTable of Contents\nIntroduction  ............................................................................................................................ 4\nTo Be, Or Not To Be An Entrepreneur  .................................................................................. 7\nYour Business Idea  ......................................................................................

In [ ]:
# We need to split the text using Character Text Split such that it should not increase token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
from langchain_cohere import CohereEmbeddings
from cohere import Client
from google.colab import userdata
import os

os.environ["COHERE_API_KEY"] = userdata.get('Cohere_API_Key')
# Access the API key
api_key = os.environ["COHERE_API_KEY"]

cohere_client = Client(api_key)

embeddings = cohere_client.embed(texts=texts).embeddings

In [ ]:
import pinecone

os.environ["PINECONE_API_KEY"] = userdata.get('Pinecone_API_Key')
Api_key = os.environ["PINECONE_API_KEY"]

pc=pinecone.Pinecone(Api_key)

index = pc.Index('document-search-index')

batch_size = 100
for i in range(0, len(texts), batch_size):
    upsert_data = [(str(j), embeddings[j]) for j in range(i, min(i + batch_size, len(texts)))]
    index.upsert(vectors=upsert_data)

In [ ]:
def query_qa_bot(question):

    question_embedding = cohere_client.embed(texts=[question]).embeddings[0]
    query_response = index.query(vector=[question_embedding], top_k=5)

    # Extract the top documents
    top_docs = [match['id'] for match in query_response['matches']]
    relevant_texts = [texts[int(doc_id)] for doc_id in top_docs]

    context = " ".join(relevant_texts)
    answer = cohere_client.generate(
        prompt=f"Answer the question based on the context: {context}\n\nQuestion: {question}\nAnswer:",
        max_tokens=150
    ).generations[0].text.strip()


    return answer

In [ ]:
sample_queries = [
    "What are the initial steps to start a business?",
    "How do I register my business?",
    'Who can start a business?'
    # "What are common business structures?",
]

for query in sample_queries:
    answer = query_qa_bot(query)
    print(f"Question: {query}\nAnswer: {answer}\n")

Question: What are the initial steps to start a business?
Answer: Some of the initial steps to start a business include: deciding on the business type, registering the company name, and registering for taxes and government programs such as GST/PST/HST and WCB/Worksafe. Additionally, it is important to get a local business license and any necessary permits and licenses to ensure compliance with local regulations.  It is important to thoroughly research the business and market before starting the business. It is also important to have the necessary skills and investment capital to start the business.  It is also important to remember that there will be paperwork to complete, and the assistance of professionals such as lawyers and accountants may be required and should be factored into start-up costs.  It is also important to be aware of global business

Question: How do I register my business?
Answer: The best place to start the process of registering your business and discovering all th